# Ensemble Challenge

Timepoint: July 15, 2021. 

Setting: New York State upon the arrival of the Delta variant. Vaccines available

In [ ]:
# TODO: 
# - collect and process data
# - collect 3 candidate models
# - set parameter values and interventions
# - create observables for cumulative cases, hospitalizations, and deaths
# - calibrate models independently
# - calibrate ensemble of 3 models 
# - improve calibration with more data
# - plot and post-process results

### Load dependencies

In [1]:
import pandas as pd

# Data processing

In [4]:
location = "New York"
BETTER_NAMES = {'Cases': 'Infected',
                'Hospitalizations': 'Hospitalized',
                'Deaths': 'Dead'}
## instantiate, making sure no dates are skipped between start and end (could be found dynamically)
full_dataset = pd.DataFrame({'date':pd.date_range(start='1/22/2020', end='3/29/2024')})

## fill the instantiated dataset
for i in ['Cases','Hospitalizations','Deaths']:
    all_data = pd.read_csv(f'https://media.githubusercontent.com/media/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20{i}.csv')
    subset   = all_data[all_data.location_name == location].groupby("date")["value"].sum().reset_index()
    subset.date = pd.to_datetime(subset.date)
    subset['cumsum'] = subset.value.cumsum()
    full_dataset = full_dataset.merge(subset[['date','cumsum']], how='outer', on='date').rename(columns={'cumsum':BETTER_NAMES[i]})

full_dataset[(full_dataset.date >= '2021-05-15') & (full_dataset.date < '2021-09-02')].reset_index(drop=True).reset_index(names=['Timepoint']).drop(['date'],axis=1)

/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_34985/2089795940.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv(f'https://media.githubusercontent.com/media/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20{i}.csv')
/var/folders/z_/l_q4g1892pqd5g0p2z2f66_m0000gn/T/ipykernel_34985/2089795940.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv(f'https://media.githubusercontent.com/media/reichlab/covid19-forecast-hub/master/data-truth/truth-Incident%20{i}.csv')


,Timepoint,Infected,Hospitalized,Dead
0,0,2083623.0,134400.0,52709.0
1,1,2085477.0,134556.0,52748.0
2,2,2086836.0,134760.0,52782.0
3,3,2088361.0,134960.0,52813.0
4,4,2089698.0,135135.0,52837.0
...,...,...,...,...
105,105,2262893.0,148866.0,54159.0
106,106,2269038.0,149106.0,54199.0
107,107,2274680.0,149454.0,54230.0
108,108,2278590.0,149812.0,54254.0


In [7]:
full_dataset[(full_dataset.date >= '2021-05-15') & (full_dataset.date < '2021-06-02')]

,date,Infected,Hospitalized,Dead
479,2021-05-15,2083623.0,134400.0,52709.0
480,2021-05-16,2085477.0,134556.0,52748.0
481,2021-05-17,2086836.0,134760.0,52782.0
482,2021-05-18,2088361.0,134960.0,52813.0
483,2021-05-19,2089698.0,135135.0,52837.0
484,2021-05-20,2091342.0,135303.0,52863.0
485,2021-05-21,2092916.0,135464.0,52888.0
486,2021-05-22,2094245.0,135599.0,52905.0
487,2021-05-23,2095319.0,135742.0,52925.0
488,2021-05-24,2096280.0,135888.0,52958.0


In [3]:
def get_data_between(start='2021-06-01',end='2021-09-02'):
    return full_dataset[(full_dataset.date >= start) & (full_dataset.date < end)].reset_index(drop=True).reset_index(names=['Timepoint']).drop(['date'],axis=1)

get_data_between('2021-06-01', '2021-09-02')

,Timepoint,Infected,Hospitalized,Dead
0,0,2102869.0,136862.0,53123.0
1,1,2103269.0,136968.0,53132.0
2,2,2103768.0,137089.0,53151.0
3,3,2104539.0,137179.0,53207.0
4,4,2105375.0,137249.0,53221.0
...,...,...,...,...
88,88,2262893.0,148866.0,54159.0
89,89,2269038.0,149106.0,54199.0
90,90,2274680.0,149454.0,54230.0
91,91,2278590.0,149812.0,54254.0
